In [7]:
import pandas as pd
import numpy as np
import math 

SEED = int(math.sqrt(201401004 + 191401009))

data = pd.read_csv("movie_revenue_data.csv")
data = pd.DataFrame(data)

In [8]:
data["Rating"] = data["Rating"]/10
data[['Votes', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'WorldwideBox Office']] = np.log2(data[['Votes', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'WorldwideBox Office']])

In [9]:
data.drop('Title', axis=1, inplace=True)

In [50]:
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [51]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,47 mins 02 secs
H2O_cluster_timezone:,Asia/Istanbul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,"7 days, 1 hour and 4 minutes"
H2O_cluster_name:,H2O_from_python_Emre_199rsu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.917 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [10]:
data = h2o.H2OFrame(data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [53]:
data

Year,Rating,Votes,1,2,3,4,5,6,7,8,9,10,WorldwideBox Office
2001,0.64,16.3724,12.4886,12.1062,13.5037,14.4913,14.4038,13.3467,12.0345,10.8455,9.86728,9.9263,26.1799
1990,0.54,14.6082,10.5333,9.43254,10.7296,11.7155,12.4561,12.5073,11.6416,10.6591,9.63118,9.44087,26.0936
1990,0.64,14.1625,11.5793,10.5137,11.3934,11.6684,11.4126,10.7507,10.1874,9.66356,9.31061,9.66178,24.352
1990,0.58,14.82,10.2228,9.37287,11.0519,12.3821,13.0626,12.62,11.3603,10.1163,8.93369,9.04166,24.996
1990,0.66,13.8169,9.49586,9.60918,11.151,12.1196,11.7674,10.4988,9.0954,7.91886,7.26679,8.35755,22.8057
1990,0.59,15.356,10.643,9.95855,11.5579,12.9179,13.7186,13.1349,11.7296,10.4818,9.46352,9.18982,27.1938
1990,0.65,16.1008,11.5479,11.411,13.25,14.3261,14.2258,13.077,11.7275,10.5517,9.56605,9.69697,25.6651
1990,0.78,17.1081,14.0918,14.5988,15.6473,14.9948,13.2506,11.4993,10.102,8.71425,8.32643,9.10329,25.6347
1990,0.74,18.7596,15.6546,15.575,16.86,17.046,15.8105,14.218,12.6678,11.4954,10.5362,10.7649,27.8687
1990,0.6,15.4545,10.9527,10.2586,11.8918,13.2007,13.7407,13.057,11.7578,10.4419,9.45121,9.4818,27.0474


In [11]:
predictors = data.columns[:-1]

In [12]:
train, test = data.split_frame(ratios = [0.8], seed = SEED)

In [56]:
train

Year,Rating,Votes,1,2,3,4,5,6,7,8,9,10,WorldwideBox Office
2001,0.64,16.3724,12.4886,12.1062,13.5037,14.4913,14.4038,13.3467,12.0345,10.8455,9.86728,9.9263,26.1799
1990,0.54,14.6082,10.5333,9.43254,10.7296,11.7155,12.4561,12.5073,11.6416,10.6591,9.63118,9.44087,26.0936
1990,0.64,14.1625,11.5793,10.5137,11.3934,11.6684,11.4126,10.7507,10.1874,9.66356,9.31061,9.66178,24.352
1990,0.58,14.82,10.2228,9.37287,11.0519,12.3821,13.0626,12.62,11.3603,10.1163,8.93369,9.04166,24.996
1990,0.66,13.8169,9.49586,9.60918,11.151,12.1196,11.7674,10.4988,9.0954,7.91886,7.26679,8.35755,22.8057
1990,0.65,16.1008,11.5479,11.411,13.25,14.3261,14.2258,13.077,11.7275,10.5517,9.56605,9.69697,25.6651
1990,0.78,17.1081,14.0918,14.5988,15.6473,14.9948,13.2506,11.4993,10.102,8.71425,8.32643,9.10329,25.6347
1990,0.74,18.7596,15.6546,15.575,16.86,17.046,15.8105,14.218,12.6678,11.4954,10.5362,10.7649,27.8687
1990,0.6,15.4545,10.9527,10.2586,11.8918,13.2007,13.7407,13.057,11.7578,10.4419,9.45121,9.4818,27.0474
1990,0.57,13.9251,8.67948,8.21432,10.0688,11.3509,12.0196,11.783,10.817,9.84862,8.58496,8.55459,22.9704


In [64]:
aml = H2OAutoML(max_runtime_secs = 900,
                seed = SEED,
                project_name = 'movie-regression',
                stopping_metric = "RMSE",
                sort_metric = "RMSE")

In [65]:
%time aml.train(x = predictors, y = 'WorldwideBox Office', training_frame = train)


AutoML progress: |█
15:06:14.890: AutoML: XGBoost is not available; skipping it.
15:20:15.942: New models will be added to existing leaderboard movie-regression@@WorldwideBox_Office (leaderboard frame=null) with already 48 models.
15:20:15.942: AutoML: XGBoost is not available; skipping it.
15:20:16.555: StackedEnsemble_BestOfFamily_4_AutoML_3_20221130_152015 [StackedEnsemble best_of_family_1 (built with AUTO metalearner, using top model from each algorithm type)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't set when building the models, or the frame was deleted.
15:20:20.958: StackedEnsemble_BestOfFamily_5_AutoML_3_20221130_152015 [StackedEnsemble best_of_family_2 (built with AUTO metalearner, using top model from each algorithm type)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictio

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_2_AutoML_4_20221130_152117_model_10


Status of Neuron Layers: predicting WorldwideBox Office, regression, gaussian distribution, Quadratic loss, 751 weights/biases, 13,4 KB, 14.466.670 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate               rate_rms               momentum    mean_weight             weight_rms          mean_bias             bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ----------------------  ---------------------  ----------  ----------------------  ------------------  --------------------  -----------------------
    1        13       Input             0.0
    2        50       RectifierDropout  30.0       0.0   0.0   0.0011876939762553846   0.0014038472436368465  0.0         0.017740574681794477    0.4197244644165039  -0.09639643160276655  0.32903504371643066
    3        1        Linear                       0.0   0.0   0.00042409284505993126  0.0004809176316484809  0.0         -0.0006834666430950165  0.9296894073486328  -0.17802437614774436  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 4.908958727833106
RMSE: 2.215617008382339
MAE: 1.5669407883813187
RMSLE: 0.1065733900484093
Mean Residual Deviance: 4.908958727833106

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 5.251687178782351
RMSE: 2.2916559904973415
MAE: 1.635939938656477
RMSLE: 0.11000063766798525
Mean Residual Deviance: 5.251687178782351

Cross-Validation Metrics Summary: 
                        mean      sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  --------  ----------  ------------  ------------  ------------  ------------  ------------
mae                     1.64005   0.10156     1.48198       1.6761        1.72059       1.72255       1.59905
mean_residual_deviance  5.25454   0.628792    4.44871       5.46234       5.86085       5.76096       4.73986
mse                     5.25454   0.628792    4.44871       5.46234       5.86085       5.76096       4.73986
r2                      0.565916  0.0298471   0.577581      0.596192      0.556737      0.518796      0.580273
residual_deviance       5.25454   0.628792    4.44871       5.46234       5.86085       5.76096       4.73986
rmse                    2.28892   0.138685    2.1092        2.33717       2.42092       2.4002        2.17712
rmsle                   0.109655  0.00957452  0.0992449     0.113641      0.118758      0.117148      0.0994852

Scoring History: 
    timestamp            duration          training_speed    epochs    terations    samples      training_rmse    training_deviance    training_mae    training_r2
--  -------------------  ----------------  ----------------  --------  -----------  -----------  ---------------  -------------------  --------------  -------------
    2022-11-30 15:30:48  0.000 sec                           0         0            0            nan              nan                  nan             nan
    2022-11-30 15:30:48  9 min  8.370 sec  864714 obs/sec    10        1            60530        2.45062          6.00552              1.75126         0.50461
    2022-11-30 15:30:53  9 min 13.414 sec  1195917 obs/sec   1010      101          6.11353e+06  2.2426           5.02924              1.62854         0.585142
    2022-11-30 15:30:58  9 min 18.419 sec  1238725 obs/sec   2070      207          1.25297e+07  2.2166           4.9133               1.58017         0.594705
    2022-11-30 15:31:00  9 min 19.870 sec  1251009 obs/sec   2390      239          1.44667e+07  2.21562          4.90896              1.56694         0.595064

Variable Importances: 
variable    relative_mportance    scaled_mportance    percentage
----------  --------------------  ------------------  ------------
Votes       1                     1          

In [116]:
lb = aml.leaderboard
lb.head()

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
DeepLearning_grid_2_AutoML_4_20221130_152117_model_10,2.29166,5.25169,1.63594,0.110001,5.25169
DeepLearning_grid_1_AutoML_2_20221130_150614_model_10,2.29239,5.25507,1.63297,0.110238,5.25507
DeepLearning_grid_1_AutoML_2_20221130_150614_model_8,2.31396,5.35443,1.65865,0.110395,5.35443
DeepLearning_grid_2_AutoML_4_20221130_152117_model_8,2.3323,5.43965,1.65369,0.111829,5.43965
StackedEnsemble_BestOfFamily_3_AutoML_2_20221130_150614,2.33712,5.46215,1.67287,0.11148,5.46215
StackedEnsemble_AllModels_2_AutoML_2_20221130_150614,2.33896,5.47073,1.67371,0.11152,5.47073
DeepLearning_grid_2_AutoML_4_20221130_152117_model_6,2.33956,5.47354,1.66741,0.112488,5.47354
DeepLearning_grid_1_AutoML_2_20221130_150614_model_6,2.34031,5.47707,1.67074,0.112599,5.47707
StackedEnsemble_AllModels_1_AutoML_2_20221130_150614,2.34308,5.49001,1.67592,0.111726,5.49001
StackedEnsemble_BestOfFamily_1_AutoML_2_20221130_150614,2.34479,5.49803,1.67665,0.11185,5.49803


In [117]:
model = h2o.get_model('DeepLearning_grid_2_AutoML_4_20221130_152117_model_10')
model.model_performance(test)

ModelMetricsRegression: deeplearning
** Reported on test data. **

MSE: 5.219924510919091
RMSE: 2.284715411362888
MAE: 1.6067819143346123
RMSLE: 0.1109704787845045
Mean Residual Deviance: 5.219924510919091

In [119]:
model_path = h2o.save_model(model=model, path="/tmp/autoML_model",force=True)

In [120]:
model_path

'C:\\tmp\\autoML_model\\DeepLearning_grid_2_AutoML_4_20221130_152117_model_10'

# ------------------------------------------------------------------

In [3]:
#DeepLearning_grid_2_AutoML_4_20221130_152117_model_10 yükle
import h2o
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM AdoptOpenJDK-11.0.11+9 (build 11.0.11+9, mixed mode)
  Starting server from c:\Users\Emre\AppData\Local\Programs\Python\Python37-32\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Emre\AppData\Local\Temp\tmpsaomnjyq
  JVM stdout: C:\Users\Emre\AppData\Local\Temp\tmpsaomnjyq\h2o_Emre_started_from_python.out
  JVM stderr: C:\Users\Emre\AppData\Local\Temp\tmpsaomnjyq\h2o_Emre_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Istanbul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,"7 days, 2 hours and 25 minutes"
H2O_cluster_name:,H2O_from_python_Emre_uxcysm
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.979 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [4]:
model_path = 'C:\\tmp\\autoML_model\\DeepLearning_grid_2_AutoML_4_20221130_152117_model_10'
model = h2o.load_model(model_path)

**Aşağıdaki kodu çalıştırmadan önce yukarıda tekrar train-test datalarını oluştur**

In [13]:
predict = model.predict(test)

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


In [14]:
predict['Actual'] = test['WorldwideBox Office']

In [15]:
test_dataFrame = predict

In [16]:
test_dataFrame

predict,Actual
26.336,27.1938
24.043,24.7196
26.6234,25.092
25.1355,22.9788
25.1992,28.6602
23.2463,25.2179
21.3367,17.6457
28.0463,27.0281
26.7536,25.4873
22.0398,24.6532


In [17]:
#sort test_dataFrame by Actual
test_dataFrame = test_dataFrame.sort('Actual')

In [18]:
test_dataFrame = pd.DataFrame(test_dataFrame.as_data_frame())

In [25]:
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
df = test_dataFrame
X = df['Actual']
y = df['predict']

fig = px.scatter(x=df['Actual'], y=df['predict'], labels={'x': 'ACTUAL', 'y': 'prediction'})
fig.add_shape(
    type="line", line=dict(dash='dash'),
    x0=y.min(), y0=y.min(),
    x1=y.max(), y1=y.max()
)
fig.show()